<div class="alert alert-block alert-success">
    <b><center>Elasticsearch 7.7</center></b>
    <b><center>데이터 모델링</center></b>
</div>

In [1]:
from elasticsearch import Elasticsearch, helpers

# 연결 및 클러스터 상태확인

In [2]:
es = Elasticsearch(hosts="elastic.rsnet", port=80)

In [3]:
es.cluster.health()

{'cluster_name': 'elasticsearch',
 'status': 'green',
 'timed_out': False,
 'number_of_nodes': 3,
 'number_of_data_nodes': 3,
 'active_primary_shards': 12,
 'active_shards': 24,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 0,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 100.0}

# indices 생성/삭제

In [4]:
INDEX_NAME = "movie_search"

In [5]:
index_movie = {
    "settings": {
        "number_of_shards": 5,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            "movieCd": {"type": "keyword"},
            "movieNm": {"type": "text", "analyzer": "standard"},
            "movieNmEn": {"type": "text", "analyzer": "standard"},
            "prdtYear": {"type": "integer"},
            "openDt": {"type": "integer"},
            "typeNm": {"type": "keyword"},
            "prdtStatNm": {"type": "keyword"},
            "nationAlt": {"type": "keyword"},
            "genreAlt": {"type": "keyword"},
            "repNationNm": {"type": "keyword"},
            "repGenreNm": {"type": "keyword"},
            "companies": {
                "properties": {
                    "companyCd": {"type": "keyword"},
                    "companyNm": {"type": "keyword"}
                }
            },
            "directors": {
                "properties": {
                    "peopleNm": {"type": "keyword"}
                }
            }
        }
    }
}

In [6]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(INDEX_NAME)
es.indices.create(INDEX_NAME, index_movie, ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_search'}

## 매핑확인

In [7]:
es.indices.get_mapping(INDEX_NAME)

{'movie_search': {'mappings': {'properties': {'companies': {'properties': {'companyCd': {'type': 'keyword'},
      'companyNm': {'type': 'keyword'}}},
    'directors': {'properties': {'peopleNm': {'type': 'keyword'}}},
    'genreAlt': {'type': 'keyword'},
    'movieCd': {'type': 'keyword'},
    'movieNm': {'type': 'text', 'analyzer': 'standard'},
    'movieNmEn': {'type': 'text', 'analyzer': 'standard'},
    'nationAlt': {'type': 'keyword'},
    'openDt': {'type': 'integer'},
    'prdtStatNm': {'type': 'keyword'},
    'prdtYear': {'type': 'integer'},
    'repGenreNm': {'type': 'keyword'},
    'repNationNm': {'type': 'keyword'},
    'typeNm': {'type': 'keyword'}}}}}

# 문서 생성

In [10]:
movie_data_1 = {
  "movieCd": "20173732",
  "movieNm": "살아남은 아이",
  "movieNmEn": "Last Child",
  "prdtYear": "2017",
  "openDt": "",
  "typeNm": "장편",
  "prdtStatNm": "기타",
  "nationAlt": "한국",
  "genreAlt": "드라마,가족",
  "repNationNm": "한국",
  "repGenreNm": "드라마"
}

In [11]:
es.index(INDEX_NAME, movie_data_1, id=1)

{'_index': 'movie_search',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

# 문서 조회

In [12]:
es.search({"query" : {
    "match_all": {}
}}, INDEX_NAME)

{'took': 292,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'movie_search',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_source': {'movieCd': '20173732',
     'movieNm': '살아남은 아이',
     'movieNmEn': 'Last Child',
     'prdtYear': '2017',
     'openDt': '',
     'typeNm': '장편',
     'prdtStatNm': '기타',
     'nationAlt': '한국',
     'genreAlt': '드라마,가족',
     'repNationNm': '한국',
     'repGenreNm': '드라마'}}]}}

In [13]:
es.search({"query" : {
    "match": {
        "typeNm": "단편"
    }
}}, INDEX_NAME)

{'took': 281,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [15]:
es.search({
 "size":0,
  "aggs": {
    "indices": {
      "terms": {
        "field": "_index",
        "size": 10
      }
    }
  }
}, INDEX_NAME)

{'took': 41,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'indices': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'movie_search', 'doc_count': 1}]}}}

In [ ]:
es.reindex()